# Model Performance Analyze

In [133]:
import sys
import os
sys.path.insert(0, '../../')
import numpy as np
import pandas as pd
from pathlib import Path
from ast import literal_eval
from collections import Counter
from IPython.display import display
from typing import List, Union, Callable

from cont_gen.utils.io import load_json, load_jsonl, load_pickle
from cont_gen.evaluate.token_metrics import get_point_counts
from cont_gen.evaluate.cal_metrics import safe_p_r_f1_iou_score

In [127]:
# Define some helpfule functions
class Helper:
    def __init__(self, clause_info = None, para_data = None):
        self.helpers:List[Union[ClauseInfoHelper, ParaDataHelper]] = []
        if clause_info is not None:
            self.helpers.append(ClauseInfoHelper(clause_info))
        if para_data is not None:
            self.helpers.append(ParaDataHelper(para_data))
    
    def __getattr__(self, name) -> Callable:
        for helper in self.helpers:
            if hasattr(helper, name):
                return getattr(helper, name)
        raise AttributeError

class ClauseInfoHelper:
    """Basic functions0. to show clause information"""
    def __init__(self, clause_info):
        self.clause_info = clause_info
    
    def get_cla_name(self, cla_id):
        """Return clause name based on clause id"""
        return self.clause_info.iloc[cla_id, 0]

class ParaDataHelper:
    """Retrieve based on contract title and para_idx for clean data format"""
    def __init__(self, para_data):
        self.para_data = para_data
        
        meta2para = {}
        for doc in para_data:
            title = doc['title']
            for pi, para in enumerate(doc['paras']):
                meta2para[(title, pi)] = para
        self.meta2para = meta2para
    
    def get_text(self, title, para_idx):
        return self.meta2para.get((title, para_idx), {}).get('text')
    
    def get_para(self, title, para_idx):
        return self.meta2para.get((title, para_idx), {})

In [2]:
# Load original data
seed = 42
proj_dir = Path.resolve(Path('../../'))

In [3]:
split_dir = proj_dir / f'data/ood_split/seed{seed}_tr29/llama3'
train_meta = pd.read_csv(split_dir / 'train_meta.csv', converters={'answers': literal_eval})
test_meta_id = pd.read_csv(split_dir / 'test_meta_id.csv', converters={'answers': literal_eval})
test_meta_ood = pd.read_csv(split_dir / 'test_meta_ood.csv', converters={'answers': literal_eval})

In [7]:
all_paras = load_jsonl(proj_dir / 'data/cuad_clean/merge_split/paras_llama3_512.jsonl')
raw_cuad = load_json(proj_dir / 'data/cuad_split/CUADv1.json')
clause_info = pd.read_csv(proj_dir / 'data/clause/all_info.csv')

In [128]:
helper = Helper(clause_info ,all_paras)

In [6]:
test_meta_id['q_id'].unique()

array([ 2,  3,  4,  9, 10, 11, 12, 13, 16, 18, 19, 20, 21, 22, 23, 24, 25,
       26, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 39])

In [9]:
# Get predictions
id_preds = load_jsonl(proj_dir / 'runs/ood/llama3/seed42_tr29/pmt_01_all_lr1e-5_bs16_wd0.0/checkpoint-78460/predictions_id_sampled.jsonl')
ood_preds = load_jsonl(proj_dir / 'runs/ood/llama3/seed42_tr29/pmt_01_all_lr1e-5_bs16_wd0.0/checkpoint-78460/predictions_ood_sampled.jsonl')

In [11]:
print(len(id_preds))
print(len(test_meta_id))
# The sampled test samples are thoes with type 1,2 and 3 in meta data.

41731
162371


## Analyze for each clause type

### Get Metrics for Each Sample

In [80]:
def parse_pred(text: str):
    if text == 'No' or text == 'No.':
        return ''
    else:
        text = text.removeprefix('Yes.')
        lines = [k for k in text.split('\n') if k]
        lines = [k.removeprefix('- ') for k in lines]
        return ' '.join(lines)

def get_point_metrics(row):
    pred = parse_pred(row['prediction'])
    ground = ' '.join(row['answers'])
    counts = get_point_counts(ground, pred)
    p, r, f1, iou = safe_p_r_f1_iou_score(*counts)
    return {'p': p, 'r': r, 'f1': f1, 'iou': iou}

def add_metrics(df: pd.DataFrame)->pd.DataFrame:
    met_df = pd.DataFrame(df.apply(get_point_metrics, axis = 1).tolist())
    return pd.concat([df, met_df], axis = 1)

In [81]:
print(test_meta_id.columns)
print(id_preds[0].keys())

# Merge samples and their predictions
id_preds_df = pd.DataFrame(id_preds)
print(len(id_preds_df))
id_comb = pd.merge(test_meta_id, id_preds_df, how = 'inner', on = ['title', 'para_idx', 'q_id', 'type'])
print(len(id_comb))
id_comb.head(2)

id_comb = add_metrics(id_comb)
id_comb.head(2)

Index(['title', 'para_idx', 'q_id', 'answers', 'type'], dtype='object')
dict_keys(['title', 'para_idx', 'q_id', 'type', 'prediction'])
41731
41731


,title,para_idx,q_id,answers,type,prediction,p,r,f1,iou
0,LohaCompanyltd_20191209_F-1_EX-10.16_11917878_...,3,2,[],2,No,1.0,1.0,1.0,1.0
1,LohaCompanyltd_20191209_F-1_EX-10.16_11917878_...,3,3,[],2,No,1.0,1.0,1.0,1.0


### Clause Type Performance

In [125]:
def show_para_quests(df: pd.DataFrame, title, para_idx):
    """
    Print the positive question and negative questions with positive predictions.
    """
    part = df[(df['title'] == title) & (df['para_idx'] == para_idx)]
    def ft_func(row):
        return row['type'] == 1 or (row['type'] == 2 and row['iou'] < 0.99)
    display(part[part.apply(ft_func, axis = 1)])

In [60]:
# Map from meta data to paragraph data
meta2para = {}
meta2text = {}
for doc in all_paras:
    title = doc['title']
    for pi, para in enumerate(doc['paras']):
        meta2para[(title, pi)] = para
        meta2text[(title, pi)] = para['text']

In [59]:
all_paras[0]['paras'][0].keys()

dict_keys(['text', 'offset', 'qas', 'old_para_idx'])

In [105]:
comb_df = id_comb # specify the metric-combined dataframe
q_ids = test_meta_id['q_id'].unique().tolist()

q_id = q_ids[1]
cla_name = clause_info.iloc[q_id,0]
print(cla_name)

part = comb_df[comb_df['q_id'] == q_id]
ave_iou = part['iou'].mean()

# IOU on splits: TP, tp_other (has other clause), Neg_rand
iou_tp = part[part['type'] == 1]['iou'].mean()
iou_ot = part[part['type'] == 2]['iou'].mean()
iou_rd = part[part['type'] == 3]['iou'].mean()

print(f'Total IOU: {ave_iou:.4f}\nTrue Pos: {iou_tp:.4f}, Confuse: {iou_ot:.4f}, Rand Neg: {iou_rd:.4f}')

Effective Date
Total IOU: 0.9573
True Pos: 0.5842, Confuse: 0.9692, Rand Neg: 0.9980


In [106]:
# Bad performance on true positive
part_up_tp = part[(part['type'] == 1) & (part['iou'] < 0.9)]
print(len(part_up_tp))

36


In [122]:
idx = 8
row = part_up_tp.iloc[idx].to_dict()
p_text = meta2text[(row['title'], row['para_idx'])]
print(p_text)
print(f'answers: {row["answers"]}')
print(f'prediction: {row["prediction"]}')
print(f'iou: {row["iou"]}')

 (Signature Pages Follow)
 -8-
 CITY OF FORT STOCKTON:
 CITY OF FORT STOCKTON
By: _ (Printed Name) Title: Date:
 STATE OF TEXAS § COUNTY OF PECOS §
This instrument was acknowledged before me on the day of , 2014, by Raul B. Rodriguez, City Manager of the City of Fort Stockton.
 Notary Public, State of Texas My Commission Expires
answers: ['day of , 2014']
prediction: No
iou: 0.0


In [126]:
show_para_quests(comb_df, row['title'], row['para_idx'])

,title,para_idx,q_id,answers,type,prediction,p,r,f1,iou
6753,STWRESOURCESHOLDINGCORP_08_06_2014-EX-10.1-COO...,17,2,"[day of , 2014]",1,No,1.0,0.0,0.0,0.0
6754,STWRESOURCESHOLDINGCORP_08_06_2014-EX-10.1-COO...,17,3,"[day of , 2014]",1,No,1.0,0.0,0.0,0.0


In [66]:
for i, row in clause_info.iterrows():
    print(i, row['clause_type'])

0 Document Name
1 Parties
2 Agreement Date
3 Effective Date
4 Expiration Date
5 Renewal Term
6 Notice Period To Terminate Renewal
7 Governing Law
8 Most Favored Nation
9 Non-Compete
10 Exclusivity
11 No-Solicit Of Customers
12 Competitive Restriction Exception
13 No-Solicit Of Employees
14 Non-Disparagement
15 Termination For Convenience
16 Rofr/Rofo/Rofn
17 Change Of Control
18 Anti-Assignment
19 Revenue/Profit Sharing
20 Price Restrictions
21 Minimum Commitment
22 Volume Restriction
23 Ip Ownership Assignment
24 Joint Ip Ownership
25 License Grant
26 Non-Transferable License
27 Affiliate License-Licensor
28 Affiliate License-Licensee
29 Unlimited/All-You-Can-Eat-License
30 Irrevocable Or Perpetual License
31 Source Code Escrow
32 Post-Termination Services
33 Audit Rights
34 Uncapped Liability
35 Cap On Liability
36 Liquidated Damages
37 Warranty Duration
38 Insurance
39 Covenant Not To Sue
40 Third Party Beneficiary


In [120]:
train_meta[(train_meta['q_id'] == 3) & (train_meta['type'] == 0)].iloc[0:20]

,title,para_idx,q_id,answers,type
4,LIMEENERGYCO_09_09_1999-EX-10-DISTRIBUTOR AGRE...,5,3,[The term of this Agreement shall be ten (10) ...,0
14,LIMEENERGYCO_09_09_1999-EX-10-DISTRIBUTOR AGRE...,29,3,[Unless earlier terminated otherwise provided ...,0
27,"WHITESMOKE,INC_11_08_2011-EX-10.26-PROMOTION A...",0,3,[1 August 2011],0
40,ADAMSGOLFINC_03_21_2005-EX-10.17-ENDORSEMENT A...,2,3,[The Term of this Agreement shall be for a per...,0
63,"KIROMICBIOPHARMA,INC_05_11_2020-EX-10.23-CONSU...",21,3,[This letter agreement will be effective as of...,0
66,"VEONEER,INC_02_21_2020-EX-10.11-JOINT VENTURE ...",0,3,"[October 30, 2019]",0
67,"VEONEER,INC_02_21_2020-EX-10.11-JOINT VENTURE ...",10,3,[This Amendment shall only become effective up...,0
74,"FTENETWORKS,INC_02_18_2016-EX-99.4-STRATEGIC A...",0,3,[17t h day of February 2016],0
86,DOMINIADVISORTRUST_02_18_2005-EX-99.(H)(2)-SPO...,0,3,"[February 4, 2005]",0
89,DOMINIADVISORTRUST_02_18_2005-EX-99.(H)(2)-SPO...,8,3,[This Agreement shall become effective as of t...,0


In [92]:
train_meta.iloc[248]['answers']

['This Consulting Agreement (the "Agreement") is made and entered into as of this 2nd day of January 2020,']

In [100]:
class RawCUAD:
    def __init__(self, path = proj_dir / 'data/cuad_split/CUADv1.json'):
        self.raw = load_json(path)['data']
        self.title2para = {d['title']: d['paragraphs'][0] for d in self.raw}
    
    def get_answers(self, title, cla_id):
        para = self.title2para[title]
        text = para['context']
        qa = para['qas'][cla_id]
        return text, qa


In [101]:
rcuad = RawCUAD()

In [102]:
text, qa = rcuad.get_answers(row['title'], q_id)

In [103]:
qa

{'answers': [{'text': '1st day of April, 2018 (', 'answer_start': 78}],
 'id': 'EcoScienceSolutionsInc_20180406_8-K_EX-10.1_11135398_EX-10.1_Sponsorship Agreement__Agreement Date',
 'question': 'Highlight the parts (if any) of this contract related to "Agreement Date" that should be reviewed by a lawyer. Details: The date of the contract',
 'is_impossible': False}